In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_seir_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune",
    )
    model = CompartmentalModel(
        times=(0., config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["tot_popn"] - config["infectious_seed"],
            "Infectious": config["infectious_seed"],
        }
    )
    
    # Transitions
    ave_infous = Parameter("ave_infous")
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("r0") / ave_infous,
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("ave_preinfous"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / ave_infous,
        source="Infectious", 
        dest="Immune"
    )
    
    # Outputs
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    model.request_output_for_flow(
        name="infection", 
        flow_name="infection"
    )
    
    return model

In [ ]:
model_config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 200.,
}

parameters = {
    "r0": 2.,
    "ave_infous": 2.,
    "ave_preinfous": 2.,
}

In [ ]:
seir_model = build_seir_model(model_config)
seir_model.run(parameters)
compartment_values = seir_model.get_outputs_df()

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(
    seir_model.times, 
    compartment_values["Infectious"], 
    color="red",
)
left_ax.set_xlim(0., 1e2)
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_xlabel("Time (days)")
left_ax.set_ylim(0., 2e4)
left_ax.set_yticks(range(0, 25000, 5000))
left_ax.yaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_ylabel("Number of infectious persons")
left_ax.spines.top.set_visible(False)
left_ax.spines.right.set_visible(False)

image = plt.imread("./figures/model_3_1_fig_flu.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

In [ ]:
parameters = {
    "r0": 13.,
    "ave_preinfous": 8.,
    "ave_infous": 7.,
}
seir_model.run(parameters=parameters)

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(
    seir_model.times, 
    seir_model.get_outputs_df()["Infectious"], 
    color="red",
)
left_ax.set_xlim(0., 1e2)
left_ax.xaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_xlabel("Time (days)")
left_ax.set_ylim(0., 5e4)
left_ax.set_yticks(range(0, 60000, 10000))
left_ax.yaxis.set_minor_locator(AutoMinorLocator(5))
left_ax.set_ylabel("Number of infectious persons")
left_ax.spines.top.set_visible(False)
left_ax.spines.right.set_visible(False)

image = plt.imread("./figures/model_3_1_fig_measles.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");